In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
import nltk
import re
  
# for Stemming propose  
from nltk.stem.porter import PorterStemmer 
from nltk.corpus import stopwords 

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

stock_data = pd.read_csv("input/DJIA_table.csv") 
news_data = pd.read_csv("input/RedditNews.csv") 
news_data = news_data.merge(stock_data, on='Date')

train_news_data = news_data[news_data['Date'] < '2015-01-01']
test_news_data = news_data[news_data['Date'] > '2014-12-31']

# Any results you write to the current directory are saved as output.

In [ ]:
grouped_news = pd.DataFrame(columns=['Date','Stacked News','Open','High','Low','Close','Volume','Adj Close'])
grouped_news['Date'] = train_news_data.groupby('Date')['Date'].apply(set)
grouped_news['Stacked News'] = train_news_data.groupby('Date')['News'].apply(lambda x: x.sum())
grouped_news['Open'] = train_news_data.groupby('Date')['Open'].unique()
grouped_news['High'] = train_news_data.groupby('Date')['High'].unique()
grouped_news['Low'] = train_news_data.groupby('Date')['Low'].unique()
grouped_news['Close'] = train_news_data.groupby('Date')['Close'].unique()
grouped_news['Volume'] = train_news_data.groupby('Date')['Volume'].unique()
grouped_news['Adj Close'] = train_news_data.groupby('Date')['Adj Close'].unique()

In [ ]:
# Remove all the special characters
grouped_news['Stacked News'] = [re.sub(r'\W', ' ', row) for row in grouped_news['Stacked News']]

# remove all single characters
grouped_news['Stacked News'] = [re.sub(r'\s+[a-zA-Z]\s+', ' ', row) for row in grouped_news['Stacked News']]

# Remove single characters from the start
grouped_news['Stacked News'] = [re.sub(r'\^[a-zA-Z]\s+', ' ', row) for row in grouped_news['Stacked News']]

# Substituting multiple spaces with single space
grouped_news['Stacked News'] = [re.sub(r'\s+', ' ', row, flags=re.I) for row in grouped_news['Stacked News']]

# Removing prefixed 'b'
grouped_news['Stacked News'] = [re.sub(r'^b\s+', '', row) for row in grouped_news['Stacked News']]

# Removing numbers
grouped_news['Stacked News'] = [re.sub(r'\d+', '', row) for row in grouped_news['Stacked News']]

In [ ]:
grouped_news['Date'] = grouped_news['Date'].astype(str)
grouped_news['Date'] = [row.strip("{}''") for row in grouped_news['Date']]

grouped_news['Open'] = grouped_news['Open'].astype(str)
grouped_news['Open'] = [row.strip("[]") for row in grouped_news['Open']]
grouped_news['Open'] = grouped_news['Open'].astype(float)

grouped_news['High'] = grouped_news['High'].astype(str)
grouped_news['High'] = [row.strip("[]") for row in grouped_news['High']]
grouped_news['High'] = grouped_news['High'].astype(float)

grouped_news['Low'] = grouped_news['Low'].astype(str)
grouped_news['Low'] = [row.strip("[]") for row in grouped_news['Low']]
grouped_news['Low'] = grouped_news['Low'].astype(float)

grouped_news['Close'] = grouped_news['Close'].astype(str)
grouped_news['Close'] = [row.strip("[]") for row in grouped_news['Close']]
grouped_news['Close'] = grouped_news['Close'].astype(float)

grouped_news['Volume'] = grouped_news['Volume'].astype(str)
grouped_news['Volume'] = [row.strip("[]") for row in grouped_news['Volume']]
grouped_news['Volume'] = grouped_news['Volume'].astype(float)

grouped_news['Adj Close'] = grouped_news['Adj Close'].astype(str)
grouped_news['Adj Close'] = [row.strip("[]") for row in grouped_news['Adj Close']]
grouped_news['Adj Close'] = grouped_news['Adj Close'].astype(float)

In [ ]:
grouped_news['Stacked News'] = [row.lower() for row in grouped_news['Stacked News']]
grouped_news['Vectorized Words'] = [CountVectorizer().build_tokenizer()(row) for row in grouped_news['Stacked News']]

In [ ]:
ps = PorterStemmer() 
for itr,row in enumerate(grouped_news['Vectorized Words']):
    a = ''
    for inneritr, word in enumerate(row):
        if not word in set(stopwords.words('english')):
            a = a + ' ' + ps.stem(word)
    grouped_news['Vectorized Words'].iloc[itr] = a
grouped_news['Vectorized Words'] = [CountVectorizer().build_tokenizer()(row) for row in grouped_news['Vectorized Words']]

In [ ]:
grouped_news['Word Count'] = [[[x,data.count(x)] for x in set(data)] for data in grouped_news['Vectorized Words']]

In [ ]:
grouped_news['Increase/Decrease'] = grouped_news['Close'].diff()
grouped_news.dropna(inplace=True)